In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.losses import MeanAbsoluteError
from keras.optimizers import Adam
from keras.layers import InputLayer
from keras.models import load_model
from tensorflow import keras
from pandas import DataFrame
from pandas import concat

from dateutil.relativedelta import relativedelta
from datetime import datetime

from statistics import mean

from tensorflow.keras import layers
import keras_tuner as kt

from keras.layers import concatenate

from tensorflow.keras.layers import Input, Concatenate, Dense, Reshape
from tensorflow.keras.models import Model

import tensorflow as tf
import keras as ks

from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter



In [ ]:
# =============================================================================
# Import Data
# =============================================================================

data = pd.read_csv('Norway-NA-15_$47$_9-F-9 A depth.csv')
#data = data.iloc[1817:10000,:]
data = data.iloc[1424:10080,:] #470 to 848 + 10

In [ ]:
# =============================================================================
# NaN Values Count
# =============================================================================
'''
Na=data.isna().sum(axis=0)
NaPercent = Na*100/8183
NaPercent.value_counts()
count = NaPercent>95
count.sum()
'''

In [ ]:
# =============================================================================
# Variance Count
# =============================================================================
'''
var = data.var(axis=0)
varmask = var > 0.1
varcount = varmask.value_counts()
'''

In [ ]:
# =============================================================================
# Droping the Unwanted Columns
# =============================================================================

dataFiltered = data.drop(['MWD Magnetic Toolface dega',
                          'IMP/ARC Attenuation Conductivity 40-in. at 2 MHz mS/m',
                          'ARC Annular Pressure kPa',
                          'MWD Collar RPM rpm',
                          'IMP/ARC Non-BHcorr Phase-Shift Resistivity 28-in. at 2 MHz ohm.m',
                          'IMP/ARC Phase-Shift Conductivity 40-in. at 2 MHz mS/m',
                          'Annular Temperature degC',
                          'IMP/ARC Non-BHcorr Phase-Shift Resistivity 40-in. at 2 MHz ohm.m',
                          'ARC Gamma Ray (BH corrected) gAPI',
                          'IMP/ARC Non-BHcorr Attenuation Resistivity 40-in. at 2 MHz ohm.m',
                          'MWD Stick-Slip PKtoPK RPM rpm',
                          'IMP/ARC Non-BHcorr Attenuation Resistivity 28-in. at 2 MHz ohm.m',
                          'IMP/ARC Phase-Shift Conductivity 28-in. at 2 MHz mS/m',
                          'MWD Total Shocks unitless',
                          'ROPIH s/m',
                          'HKLO kkgf',
                          'DRET unitless',
                          'Pump 1 Strokes unitless',
                          'Pump 2 Strokes unitless',
                          'Pump 3 Strokes unitless',
                          'Total Strokes unitless',
                          'Bit Depth m',
                          'String weight (rot,avg) kkgf',
                          'TOBO s',
                          'HKLI kkgf',
                          'Elapsed time in-slips s',
                          'Pump 4 Strokes unitless',
                          'Mud Density Out g/cm3',
                          'Iso-butane (IC4) ppm',
                          'Nor-butane (NC4) ppm',
                          'Lag Depth (TVD) m',
                          'Ethane (C2) ppm',
                          'TMP In degC',
                          'n-Penthane ppm',
                          'Iso-pentane (IC5) ppm',
                          'Gas (avg) %',
                          'Propane (C3) ppm',
                          'Temperature Out degC',
                          'Methane (C1) ppm',
                          'Unnamed: 0.1',
                          'Unnamed: 0',
                          'nameWellbore',
                          'name',
                          'Bit run number unitless',
                          'Inverse ROP (5ft avg) s/m',
                          'Inverse ROP s/m',
                          'Rate of Penetration (5ft avg) m/h',
                          'Rate of penetration m/h',

                          'PowerUP Shock Rate 1/s',
                          'EDRT unitless',
                          'MWD Shock Risk unitless',
                          'Bit run number unitless',
                          'Pump 4 Stroke Rate 1/min',
                          'Rig Mode unitless',
                          'AJAM_MWD unitless',
                          'BHFG unitless',
                          'SHK3TM_RT min',
                          'Mud Density In g/cm3.1',
                          'IMWT g/cm3',
                          'Mud Density In g/cm3',
                          'SPN Sp_RigMode 2hz unitless',
                          'Pump 2 Stroke Rate 1/min',
                          'Pass Name unitless'],axis=1)

In [ ]:
# =============================================================================
# Data Imputation
# =============================================================================

dataImputed = dataFiltered.interpolate(method='linear', axis=0)
dataImputed = dataImputed.fillna(method='backfill', axis=0)
dataImputed = dataImputed.fillna(method='ffill', axis=0)

In [ ]:
# =============================================================================
# Index Set to Measured Depth
# =============================================================================

dataImputed = dataImputed.set_index('Measured Depth m')

In [ ]:
# =============================================================================
# Data Resampling
# =============================================================================

DataResampled = dataImputed.reset_index()
MD = DataResampled.iloc[:,0].values
RestOfData = DataResampled.iloc[:,:].values

desired_depth_step = 0.05
min_depth = 500 #np.min(MD)
max_depth = 845 #np.max(MD)
new_depths = np.arange(min_depth, max_depth + desired_depth_step, desired_depth_step)
new_depths = np.round(new_depths, decimals=2)

num_measurements = 53

new_measurement_data = []
for i in range(num_measurements):
    new_measurement_data.append(np.interp(new_depths, MD, RestOfData[:, i],
                                          left=None, right=None, period=None))

new_measurement_data = np.array(new_measurement_data).T



new_measurement_data[:,0] = np.round(new_measurement_data[:, 0], decimals=2)

Col = DataResampled.columns.tolist()

DataResampled = pd.DataFrame(data=new_measurement_data,columns=Col)
DataResampled.set_index('Measured Depth m',inplace=True)

DataResampled = DataResampled[470:889] #reconsider this


rolling_window = 10
DataSmoothed = DataResampled.rolling(window=rolling_window, min_periods=1, center=True).mean()

In [ ]:
# =============================================================================
# Features and Targets
# =============================================================================

dataROP = DataSmoothed[500:844.01]

Inclination = DataSmoothed['MWD Continuous Inclination dega']
RateOfPenetration = dataROP['Rate of Penetration m/h']
Inc = dataROP['MWD Continuous Inclination dega']

Inclination = Inclination.to_frame()
RateOfPenetration= RateOfPenetration.to_frame()
Inc= Inc.to_frame()


FeaturesROP = dataROP.drop(['Rate of Penetration m/h',
                            'MWD Continuous Inclination dega'], axis=1)

In [ ]:
# =============================================================================
# Inclination Data Resampling
# =============================================================================

InclinationData = Inclination.round({"MWD Continuous Inclination dega":2})

InclinationData = InclinationData[470:889]

In [ ]:
# =============================================================================
#  Scaling
# =============================================================================


FeaturesROP = (FeaturesROP-FeaturesROP.min())/(
    FeaturesROP.max()-FeaturesROP.min())

In [ ]:
# =============================================================================
# Inclination Data Preparation
# =============================================================================


'''Creating the dataset (Forcasting --> Supervised Learning)'''


LearningWindow = 100
ForwardPrediction = 1

def series_to_supervised (data, n_in, n_out, dropnan):

    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(),list()
    # input sequence (t-n,...,t-1)
    for i in range (n_in,0,-1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1,i)) for j in range (n_vars)]
    # output sequence (t,...,t+n)
    for i in range (0,n_out):
        cols.append(df.shift(-i))
        if i==0:
            names += [('var%d(t)' % (j+1)) for j in range (n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1,i)) for j in range (n_vars)]

    # Put it togather
    agg = concat(cols,axis=1)
    agg.columns = names

    #Drop rows aith NaN values
    if dropnan:
        agg.dropna(inplace=True)

    return agg


# Inclination Change
#der = InclinationData.diff()
#der = der.fillna(method='backfill', axis=0)

XY = series_to_supervised(InclinationData,
                          LearningWindow,
                          ForwardPrediction,
                          dropnan=True)
XY = XY[500:844]

X2, y2 = XY.iloc[:,:LearningWindow], XY.iloc[:,LearningWindow:]

In [ ]:
# =============================================================================
# Multi Target Data Preparation
# =============================================================================

X1= FeaturesROP
y1= RateOfPenetration

y = pd.concat([RateOfPenetration,Inc],axis=1)

In [ ]:
# =============================================================================
# Exporting to Excel to be used in google colab
# =============================================================================

#X1
file_name_X1 = 'X1.csv'
X1.to_csv(file_name_X1)
#y1
file_name_y1 = 'y1.csv'
y1.to_csv(file_name_y1)
#X2
file_name_X2 = 'X2.csv'
X2.to_csv(file_name_X2)
#y2
file_name_y2 = 'y2.csv'
y2.to_csv(file_name_y2)

In [ ]:
# =============================================================================
# Data Spliting
# =============================================================================

X1_train, y1_train = X1[500:779.99], y1[500:779.99]
X1_val, y1_val = X1[780:819.99], y1[780:819.99]
X1_test, y1_test = X1[820:844], y1[820:844]


X2_train, y2_train = X2[500:779.99], y2[500:779.99]
X2_val, y2_val = X2[780:819.99], y2[780:819.99]
X2_test, y2_test = X2[820:844], y2[820:844]


y_train,y_val,y_test = y[500:779.99],y[780:819.99],y[820:844]

In [ ]:
# =============================================================================
# Multi Target Model
# =============================================================================

drop=0.05

modelMulti = Sequential ()

modelMulti.add(Dense(1024, input_dim=50, activation='relu'))
modelMulti.add(Dropout(drop))


modelMulti.add(Dense(1024, activation='relu'))
modelMulti.add(Dropout(drop))


modelMulti.add(Dense(1, activation='linear'))

#creating a checkpoint when the best validation loss is found
cpMulti = ModelCheckpoint('modelMulti/', save_best_only=True)

modelMulti.compile(loss='mse',
                   optimizer='adam',
                   metrics='mse')
modelMulti.summary()

historyMulti = modelMulti.fit(X1_train, y1_train,
                          validation_data=(X1_val,y1_val),
                          callbacks=[cpMulti],
                          epochs=50)

#calling back the best model from the checkpoint
modelMulti = load_model('modelMulti/')


'''Loss Plots'''
lossMulti = historyMulti.history['loss']
val_lossMulti = historyMulti.history['val_loss']
epochs = range(1,len(lossMulti)+1)
plt.figure(figsize=(10,6),dpi=500)
plt.plot(epochs,lossMulti, 'y', label='Training loss',linewidth=2)
plt.plot(epochs,val_lossMulti, 'r', label='Validation loss',linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
# =============================================================================
# LSTM Inclination Model
# =============================================================================


#Inclination Prediction Model
ModelS = Sequential()


ModelS.add(LSTM(512,#batch_input_shape=(5,30,1),
                input_shape=(100,1),
                stateful=False,name="LSTM1"))
#ModelS.add(Dropout(0.05,name="DROP2"))

ModelS.add(Dense(128, activation='relu',name="DENSE1"))
ModelS.add(Dropout(0.05,name="DROP1"))

ModelS.add(Dense(1,name="DENSE2"))

ModelS.summary()

#creating a checkpoint when the best validation loss is found
cpS = ModelCheckpoint('ModelS/', save_best_only=True)

ModelS.compile (loss='mse',
                optimizer='adam',
                metrics='mse')

historyInc = ModelS.fit(X2_train,y2_train,#batch_size=5,
                        callbacks=[cpS],
                        validation_data=(X2_val,y2_val),
                        epochs=50)

#Calling the Best Performing Model
ModelS = load_model('ModelS/')

''' Loss Plots '''
lossInc = historyInc.history['loss']
val_lossInc = historyInc.history['val_loss']
epochs = range(1,len(lossInc)+1)
font = {'family':'times new roman',
        'weight':'normal',
        'size':15}
plt.rc('font', **font)
plt.figure(figsize=(10,6),dpi=500)
plt.plot(epochs,lossInc, 'y', label='Training loss',linewidth=2)
plt.plot(epochs,val_lossInc, 'r', label='Validation loss',linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

In [ ]:
# =============================================================================
# Predictions Of the Multi Target Model
# =============================================================================

Train_Prediction_1 = modelMulti.predict(X1_train)
Val_Prediction_1 = modelMulti.predict(X1_val)

Test_Prediction_1 = modelMulti.predict(X1_test)

mseROP = mean_squared_error(y1_test.iloc[:,0], Test_Prediction_1[:,0])
maeROP = mean_absolute_error(y1_test.iloc[:,0], Test_Prediction_1[:,0])

mseINC = mean_squared_error(y1_test.iloc[:,1], Test_Prediction_1[:,1])
maeINC = mean_absolute_error(y1_test.iloc[:,1], Test_Prediction_1[:,1])

plt.plot(y1_test.values)
plt.plot(Test_Prediction_1)

In [ ]:
# =============================================================================
# Predictions of the LSTM Model
# =============================================================================

Train_Prediction_2 = ModelS.predict(X2_train)
Val_Prediction_2 = ModelS.predict(X2_val)

Test_Prediction_2 = ModelS.predict(X2_test)

mseS = mean_squared_error(y2_test, Test_Prediction_2)
maeS = mean_absolute_error(y2_test, Test_Prediction_2)

plt.plot(y2_test.values)
plt.plot(Test_Prediction_2)

In [ ]:
# =============================================================================
# LSTM and MLP Model Merging
# =============================================================================

OutMLP = modelMulti.output
OutLSTM = ModelS.output

concatenated_output = Concatenate(name='concatenated_output')([OutMLP ,OutLSTM])

dense_layer1 = Dense(1024, activation='relu')(concatenated_output)

dense_layer2 = Dense(128, activation='relu')(dense_layer1)

final_output = Dense(2, activation='linear')(dense_layer2)

combined_model = Model(inputs=[modelMulti.input,ModelS.input],
                       outputs=final_output)

cpAll = ModelCheckpoint('combined_model/', save_best_only=True)

combined_model.compile(loss='mse', optimizer='adam')

combined_model.summary()

#keras.utils.plot_model(combined_model,show_shapes=False,
#                       show_layer_names=False,dpi=100)


historyAll = combined_model.fit([X1_train, X2_train],y_train,
                                validation_data=([X1_val,X2_val],y_val),
                                callbacks=[cpAll],
                                epochs=10)

'''
# First Concatenation

outS = Reshape((1,))(modelMulti.output[:,1])
outROP = Reshape((1,))(modelMulti.output[:,0])

concatenated_output1 = Concatenate(name='concatenated_output')([outS, ModelS.output])

dense_layer1 = Dense(1024, activation='relu')(concatenated_output1)
final_output1 = Dense(1, activation='linear')(dense_layer1)

combined_model_S = Model(inputs=[modelMulti.input,ModelS.input],
                       outputs=final_output1)

combined_model_S.compile(optimizer='adam', loss='mean_squared_error')

combined_model_S.summary()


# Second Concatenation

concatenated_output2 = Concatenate(name='concatenated_output2')([outROP, combined_model_S.output])

final_output2 = Dense(2, activation='linear')(concatenated_output2)

combined_model_All = Model(inputs=[modelMulti.input,ModelS.input],
                           outputs=final_output2)

cpAll = ModelCheckpoint('combined_model_All/', save_best_only=True)

combined_model_All.compile(optimizer='adam', loss='mean_squared_error')

combined_model_All.summary()


#keras.utils.plot_model(combined_model_All,show_shapes=False,
#                       show_layer_names=False,dpi=100)


historyAll = combined_model_All.fit([X1_train, X2_train],
                                    y1_train,
                                    validation_data=([X1_val,X2_val],y1_val),
                                    callbacks=[cpAll],
                                    epochs=100)
'''


''' Loss Plots '''
lossAll = historyAll.history['loss']
val_lossAll = historyAll.history['val_loss']
epochs = range(1,len(lossAll)+1)
font = {'family':'times new roman',
        'weight':'normal',
        'size':15}
plt.rc('font', **font)
plt.figure(figsize=(10,6),dpi=500)
plt.plot(epochs,lossAll, 'y', label='Training loss',linewidth=2)
plt.plot(epochs,val_lossAll, 'r', label='Validation loss',linewidth=2)
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()


combined_model = load_model('combined_model/')

In [ ]:
# =============================================================================
# Tests
# =============================================================================

Train_Prediction = combined_model.predict([X1_train,X2_train])
Val_Prediction = combined_model.predict([X1_val,X2_val])

Test_Prediction = combined_model.predict([X1_test,X2_test])


mseROP = mean_squared_error(y_test.iloc[:,0], Test_Prediction[:,0])
maeROP = mean_absolute_error(y_test.iloc[:,0], Test_Prediction[:,0])

mseINC = mean_squared_error(y_test.iloc[:,1], Test_Prediction[:,1])
maeINC = mean_absolute_error(y_test.iloc[:,1], Test_Prediction[:,1])

plt.plot(y2_test.values)
plt.plot(Test_Prediction[:,1])

In [ ]:
# =============================================================================
# Preparing Data For Plots
# =============================================================================

RealIndex = np.arange(start=500,stop=845,step=0.05)
trainIndex = np.arange(start=500,stop=780,step=0.05)
valIndex = np.arange(start=780,stop=820,step=0.05)
testIndex = np.arange(start=820,stop=845,step=0.05)

IncReal = InclinationData[500:844]
IncReal = IncReal.reset_index()
IncReal = IncReal.iloc[:,1]


IncReal = pd.DataFrame(data={'index':RealIndex,
                              'Real Inclination':IncReal})
IncReal.set_index('index',inplace=True)



train_results_ROP = pd.DataFrame(data={'index':trainIndex,
                                       'Train Predictions ROP':Train_Prediction[:,0],
                                       'Actual ROP':y1_train})
train_results_ROP.set_index('index',inplace=True)

train_results_Inc = pd.DataFrame(data={'index':trainIndex,
                                   'Train Predictions Inclination':Train_Prediction[:,1],
                                   'Actual inclination':y2_train})
train_results_Inc.set_index('index',inplace=True)



val_results_ROP = pd.DataFrame(data={'index':valIndex,
                                   'Validation Predictions ROP':Val_Prediction[:,0],
                                   'Actual ROP':y1_val})
val_results_ROP.set_index('index',inplace=True)

val_results_Inc = pd.DataFrame(data={'index':valIndex,
                                   'Validation Predictions Inclination':Val_Prediction[:,1],
                                   'Actual inclination':y2_val})
val_results_Inc.set_index('index',inplace=True)



test_results_ROP = pd.DataFrame(data={'index':testIndex,
                                   'Test Predictions ROP':Test_Prediction[:,0],
                                   'Actual ROP':y1_test})
test_results_ROP.set_index('index',inplace=True)

test_results_Inc = pd.DataFrame(data={'index':testIndex,
                                   'Test Predictions Inclination':Test_Prediction[:,1],
                                   'Actual inclination':y2_test})
test_results_Inc.set_index('index',inplace=True)

In [ ]:
# =============================================================================
# Plotting
# =============================================================================

font = {'family':'times new roman',
        'weight':'normal',
        'size':20}
plt.rc('font', **font)


# ROP Plot
plt.figure(figsize=(10,6),dpi=500)
plt.plot(RateOfPenetration,'black',linewidth=3,label='Real Inclination')
#plt.plot(test_results_ROP.iloc[:,1],linewidth=3,label='Real Inclination')
#plt.plot(test_results_ROP.iloc[:,0],linewidth=3,label='Predicted Inclination')
#plt.plot(val_results_ROP.iloc[:,1],linewidth=3,label='Real Inclination')
#plt.plot(val_results_ROP.iloc[:,0],linewidth=3,label='Predicted Inclination')
plt.xlabel('Measured Depth (m)')
plt.ylabel('Rate of Penetration (m/h)')
plt.legend(['Rate of Penetration'])
plt.show()


#Inclination Plot
plt.figure(figsize=(10,6),dpi=500)
plt.plot(Inclination,'black',linewidth=3,label='Real Inclination')
#plt.plot(test_results_Inc.iloc[:,1],linewidth=3,label='Real Inclination')
#plt.plot(test_results_Inc.iloc[:,0],linewidth=3,label='Predicted Inclination')
#plt.plot(val_results_Inc.iloc[:,1],linewidth=3,label='Real Inclination')
#plt.plot(val_results_Inc.iloc[:,0],linewidth=3,label='Predicted Inclination')
plt.xlabel('Measured Depth (m)')
plt.ylabel('Inclination (deg)')
plt.legend()
plt.show()


# ROP Full Plot
plt.figure(figsize=(10,6),dpi=500)
plt.plot(RateOfPenetration,'black', label='Real ROP',linestyle='dashed',linewidth=3)
#plt.plot(IncReal.iloc[:320,:], label='Real ROP',linewidth=3)
plt.plot(train_results_ROP.iloc[:,0], label='Training Predcitions',linewidth=3)
plt.plot(val_results_ROP.iloc[:,0],  label='Validation Prediction',linewidth=3)
plt.plot(test_results_ROP.iloc[:,0],  label='Test Predictions',linewidth=3)
plt.xlabel('Measured Depth (m)')
plt.ylabel('Rate of Penetration (m/h)')
plt.legend()
plt.show()

#Inclination Full Plot
plt.figure(figsize=(10,6),dpi=500)
plt.plot(IncReal,'black', label='Real Inclination',linestyle='dashed',linewidth=3)
#plt.plot(IncReal.iloc[:320,:], label='Real Inclination',linewidth=3)
plt.plot(train_results_Inc.iloc[:,0], label='Training Predcitions',linewidth=3)
plt.plot(val_results_Inc.iloc[:,0],  label='Validation Prediction',linewidth=3)
plt.plot(test_results_Inc.iloc[:,0],  label='Test Predictions',linewidth=3)
plt.xlabel('Measured Depth (m)')
plt.ylabel('Inclination (deg)')
plt.legend()
plt.show()

In [ ]:
###############################################################################
# =============================================================================
# Model Looping for Training Dataset Increment
# =============================================================================
###############################################################################


maeTestVecROP10 = []
maeTestVecInc10 = []
maeTest10 = []


TrVec = []
VaVec = []
TsVec = []

for i in range(100,281, 20):
    Tr = i
    Va = Tr + 40
    Ts = Va + 25
    TrVec.append(Tr)
    VaVec.append(Va)
    TsVec.append(Ts)

    ''' DataSplit'''

    X1_train, y1_train = X1[500:500+Tr-0.01], y1[500:500+Tr-0.01]
    X1_val, y1_val = X1[500+Tr:500+Va-0.01], y1[500+Tr:500+Va-0.01]
    X1_test, y1_test = X1[500+Va:500+Ts-1], y1[500+Va:500+Ts-1]

    X2_train, y2_train = X2[500:500+Tr-0.01], y2[500:500+Tr-0.01]
    X2_val, y2_val = X2[500+Tr:500+Va-0.01], y2[500+Tr:500+Va-0.01]
    X2_test, y2_test = X2[500+Va:500+Ts-1], y2[500+Va:500+Ts-1]


    ''' Prediction '''

    #MLP
    drop=0.05

    modelMulti = Sequential ()

    modelMulti.add(Dense(1024, input_dim=50, activation='relu'))
    modelMulti.add(Dropout(drop))

    modelMulti.add(Dense(1024, activation='relu'))
    modelMulti.add(Dropout(drop))

    modelMulti.add(Dense(1, activation='linear'))

    modelMulti.compile(loss='mse', optimizer='adam')

    #LSTM
    ModelS = Sequential()


    ModelS.add(LSTM(512,#batch_input_shape=(5,30,1),
                    input_shape=(100,1),
                    stateful=False,name="LSTM1"))
    #ModelS.add(Dropout(0.05,name="DROP2"))

    ModelS.add(Dense(128, activation='relu',name="DENSE1"))
    ModelS.add(Dropout(0.05,name="DROP1"))

    ModelS.add(Dense(1,activation='linear',name="DENSE2"))

    ModelS.compile (loss='mse', optimizer='adam')

    OutMLP = modelMulti.output
    OutLSTM = ModelS.output


    #concatenated_output = Concatenate(name='concatenated_output')([OutMLP ,OutLSTM])

    #dense_layer1 = Dense(1024, activation='relu')(concatenated_output)
    #dense_layer2 = Dense(1024, activation='relu')(dense_layer1)

    output_layerROP = Dense(1, name='output1')(modelMulti.output)
    output_layerInc = Dense(1, name='output2')(ModelS.output)

    #final_output = Dense(2, activation='linear')(dense_layer2)

    combined_model = Model(inputs=[modelMulti.input,ModelS.input],
                           outputs=[output_layerROP, output_layerInc])

    #cpAll = ModelCheckpoint('combined_model/', save_best_only=True)

    cpAll = ModelCheckpoint(filepath='combined_model/',
                            monitor='val_output2_loss',
                            save_best_only=True)

    #combined_model.compile(loss={'output1': 'mse', 'output2': 'mse'},
    #         loss_weights={'output1': 1, 'output2': 1})

    combined_model.compile(loss='mse', optimizer='adam')

    combined_model.summary()

    #keras.utils.plot_model(combined_model,show_shapes=False,
    #                       show_layer_names=False,dpi=100)

    historyAll = combined_model.fit([X1_train, X2_train],[y1_train,y2_train],
                                    validation_data=([X1_val,X2_val],[y1_val,y2_val]),
                                    callbacks=[cpAll],
                                    epochs=40)

    combined_model = load_model('combined_model/')

    Prediction = combined_model.predict([X1_test, X2_test])

    maeROPTest = mean_absolute_error(y1_test, Prediction[0])
    #mseROPTest = mean_squared_error(y1_test, Prediction[0])

    maeIncTest = mean_absolute_error(y2_test, Prediction[1])
    #mseIncTest = mean_squared_error(y2_test, Prediction[1])


    Predicted_ROP = pd.DataFrame(data=Prediction[0])
    Predicted_Inclination = pd.DataFrame(data=Prediction[1])
    Prediction_Combined = pd.concat([Predicted_ROP,Predicted_Inclination],axis=1)
    y_test = pd.concat([y1_test,y2_test],axis=1)
    maeTest = mean_absolute_error(y_test, Prediction_Combined)


    maeTestVecROP10.append(maeROPTest)
    maeTestVecInc10.append(maeIncTest)
    maeTest10.append(maeTest)



'''
 plt.plot(y2_test.values)
 plt.plot(Prediction[1])

 plt.plot(y1_test.values)
 plt.plot(Prediction[0])

 plt.plot(Prediction[1])
 plt.plot(Prediction[0])

In [ ]:
# =============================================================================
# Importing Results (statistics)
# =============================================================================
VaVecNp=np.array(VaVec)
indx = VaVecNp+500

maeTestVec10 = maeTestVec10.iloc[:,1]

maeTestVecAll = pd.concat([maeTestVec1,
                           maeTestVec2,
                           maeTestVec3,
                           maeTestVec4,
                           maeTestVec5,
                           maeTestVec6,
                           maeTestVec7,
                           maeTestVec8,
                           maeTestVec9,
                           maeTestVec10.
                           VaVec,],axis=1)
maeTestVecAll.index = indx
maeTestVecAll_Mean = maeTestVecAll.mean(axis=1)
maeTestVecAll_Mean_of_Mean = maeTestVecAll_Mean.mean(axis=0)
maeTestVecAll_Mean_of_Mean = pd.DataFrame({'Total Mean':[maeTestVecAll_Mean_of_Mean]*10},index=indx)
Best_of_10 = maeTestVecAll.mean(axis=1).min()


maeTestVecIncAll = pd.concat([maeTestVecInc1,
                              maeTestVecInc2,
                              maeTestVecInc3,
                              maeTestVecInc4,
                              maeTestVecInc5,
                              maeTestVecInc6,
                              maeTestVecInc7,
                              maeTestVecInc8,
                              maeTestVecInc9,
                              maeTestVecInc10],axis=1)
maeTestVecIncAll.index = indx
maeTestVecIncAll_Mean = maeTestVecIncAll.mean(axis=1)
maeTestVecIncAll_Mean_of_Mean = maeTestVecIncAll_Mean.mean(axis=0)
maeTestVecIncAll_Mean_of_Mean = pd.DataFrame({'Total Mean':[maeTestVecIncAll_Mean_of_Mean]*10},index=indx)
Best_of_10_Inc = maeTestVecIncAll.mean(axis=0).min()


maeTestVecROPAll = pd.concat([maeTestVecROP1,
                              maeTestVecROP2,
                              maeTestVecROP3,
                              maeTestVecROP4,
                              maeTestVecROP5,
                              maeTestVecROP6,
                              maeTestVecROP7,
                              maeTestVecROP8,
                              maeTestVecROP9,
                              maeTestVecROP10],axis=1)
maeTestVecROPAll.index = indx
maeTestVecROPAll_Mean = maeTestVecROPAll.mean(axis=1)
maeTestVecROPAll_Mean_of_Mean = maeTestVecROPAll_Mean.mean(axis=0)
maeTestVecROPAll_Mean_of_Mean = pd.DataFrame({'Total Mean':[maeTestVecROPAll_Mean_of_Mean]*10},index=indx)
Best_of_10_ROP = maeTestVecROPAll.mean(axis=0).min()

In [ ]:
# =============================================================================
# Statistics Plots
# =============================================================================

''' maeTestVecAll'''

font = {'family':'times new roman',
        'weight':'normal',
        'size':20}
plt.rc('font', **font)

plt.figure(figsize=(10,6),dpi=500)
plt.scatter(indx,maeTestVecAll.iloc[:,0:1],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,1:2],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,2:3],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,3:4],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,4:5],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,5:6],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,6:7],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,7:8],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,8:9],s=70)
plt.scatter(indx,maeTestVecAll.iloc[:,9:],s=70)
plt.plot(maeTestVecAll_Mean,'black',linewidth=5,label='Mean')
plt.plot(maeTestVecAll_Mean_of_Mean,'black',linewidth=3,label='Total Mean',linestyle='dashed')
plt.xlabel('Measured Depth (m)')
plt.ylabel('MAE')
plt.xticks([640,660,680,700,720,740,760,780,800,820])
plt.legend()
plt.grid()
plt.show()


''' maeTestVecIncAll'''

font = {'family':'times new roman',
        'weight':'normal',
        'size':20}
plt.rc('font', **font)
plt.figure(figsize=(10,6),dpi=500)
plt.scatter(indx,maeTestVecIncAll.iloc[:,0:1],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,1:2],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,2:3],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,3:4],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,4:5],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,5:6],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,6:7],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,7:8],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,8:9],s=70)
plt.scatter(indx,maeTestVecIncAll.iloc[:,9:],s=70)
plt.plot(maeTestVecIncAll_Mean,'black',linewidth=5,label='Mean')
plt.plot(maeTestVecIncAll_Mean_of_Mean,'black',linewidth=3,label='Total Mean',linestyle='dashed')
plt.xlabel('Measured Depth (m)')
plt.ylabel('MAE')
plt.xticks([640,660,680,700,720,740,760,780,800,820])
plt.legend()
plt.grid()
plt.show()


''' maeTestVecROPAll'''

font = {'family':'times new roman',
        'weight':'normal',
        'size':20}
plt.rc('font', **font)
plt.figure(figsize=(10,6),dpi=500)
plt.scatter(indx,maeTestVecROPAll.iloc[:,0:1],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,1:2],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,2:3],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,3:4],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,4:5],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,5:6],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,6:7],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,7:8],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,8:9],s=70)
plt.scatter(indx,maeTestVecROPAll.iloc[:,9:],s=70)
plt.plot(maeTestVecROPAll_Mean,'black',linewidth=5,label='Mean')
plt.plot(maeTestVecROPAll_Mean_of_Mean,'black',linewidth=3,label='Total Mean',linestyle='dashed')
plt.xlabel('Measured Depth (m)')
plt.ylabel('MAE')
plt.xticks([640,660,680,700,720,740,760,780,800,820])
plt.legend()
plt.grid()
plt.show()

In [ ]:
# =============================================================================
# Plotting
# =============================================================================


#Plotting DT Input
WOB = FeaturesROP['Weight on Bit kkgf'][500:665]
ARS = FeaturesROP['Average Rotary Speed rpm'][500:665]
AST = FeaturesROP['Average Surface Torque kN.m'][500:665]

font = {'family':'times new roman',
        'weight':'normal',
        'size':20}
plt.rc('font', **font)
fig,(ax1,ax2,ax3,ax4) = plt.subplots(4, sharex=True,figsize=(10,8),dpi=500,gridspec_kw={'height_ratios': [2,1,1,1]})
ax1.plot(Inclination[:640],'black',label='Real Inclination',linewidth=3)
ax1.legend()
ax1.grid(axis="x")
ax2.plot (WOB,'black',linewidth=3,label='Weight on Bit [kkgf]')
ax2.legend()
ax2.grid(axis="x")
ax3.plot (ARS,'black',linewidth=3,label='Average Rotary Speed [rpm]')
ax3.legend()
ax3.grid(axis="x")
ax4.plot (AST,'black',linewidth=3,label='Average Surface Torque [kN.m]')
ax4.legend()
ax4.grid(axis="x")
plt.xlabel('Measured Depth (m)')
#fig.suptitle('Digital Twin Inputs used for ROP Prediction')
plt.xticks([500,525,550,575,600,625,650,665])
plt.show()

In [ ]:
# =============================================================================
# Comparison
# =============================================================================

import xgboost as xgb

regressor = xgb.XGBRegressor(objective='reg:squarederror',
                             early_stopping_rounds=100,
                             n_estimators=1000,
                             learning_rate=0.001,
                             max_depth=3)

# Train the model
regressor.fit(X1_train, y1_train,eval_set=[(X1_val, y1_val)])

y_pred = regressor.predict(X1_test)
maeTestXGB = mean_absolute_error(y1_test, y_pred)